In [ ]:
from pynq import Overlay
from pynq.lib.video import *
import cv2 as cv
import sys
import numpy as np
import time
import panda as pd

In [2]:
def contrast_enh(img, p):
    chans = cv.split(img)
    for channel in chans:
        hist = cv.calcHist([channel], [0], None, [256], [0, 256])
        threshold = max(hist)*p
        
        for first in range(hist.size):
            if hist[first] > threshold:
                break;

        for last in range(255,-1,-1):
            if hist[last] > threshold:
                break;

        if last == first:
            den = 0.1
        else:
            den = last - first
            
        scale = 255 / den
        offset = first

        channel[channel < first] = 0
        channel[channel > last] = 255
        channel[(channel >= first) & (channel <= last)] = channel[(channel >= first) & (channel <= last)] * scale - offset
        
        final_image = cv.merge([chans[0], chans[1], chans[2]])
    
    return final_image

def image_sharp(image):
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

    # Apply the filter matrix to the image
    sharpened = cv.filter2D(image, -1, kernel)
    
    return sharpened

In [ ]:
base = Overlay('base.bit')
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out
hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)
hdmi_in.start()
hdmi_out.start()

In [ ]:
res_1 = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                    dtype=np.uint8)
res_2 = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width),
                    dtype=np.uint8)

contrast_time = np.array([])
sharp_time = np.array([])

In [ ]:
hdmi_in.tie(hdmi_out)

In [ ]:
video_start = time.time()

try:
    while True:

        inframe = hdmi_in.readframe()

        proc_start = time.time()
        res_1 = contrast_enh(inframe, 0.01)
        proc_mid = time.time()
        res_2 = image_sharp(res_1)
        proc_end = time.time()
        outframe = hdmi_out.newframe()

        cv2.cvtColor(res_2, cv.COLOR_GRAY2BGR,dst=outframe)
        hdmi_out.writeframe(outframe)

        contrast_time = np.append(contrast_time, (proc_mid - proc_start))
        sharp_time = np.append(sharp_time, (proc_end - proc_mid))
except KeyboardInterrupt:
    video_end = time.time()

In [ ]:
hdmi_out.close()
hdmi_in.close()

In [ ]:
tot_time = video_end - video_start
print(f'Total Time: {tot_time} seconds')

data = {'Contrast Enh': contrast_time,
        'Sharpener': sharp_time}

df = pd.DataFrame(data)
df.to_csv('DATA.txt', index=True, header=True, sep='\t')